In [ ]:
import numpy as np 
import pandas as pd 
import os

In [ ]:
def preprocess(data: pd.DataFrame):
    print(f"********* もともとのデータshape:{data.shape} *********")

    # 新規ビジネスかどうか
    data["NewExist"] = data["NewExist"].map({1: 0, 2: 1})
    # フランチャイズコードは意味なさそう
    data["FranchiseCode"] = data["FranchiseCode"].apply(
        lambda x: 0 if x in (0, 1) else 1
    )
    # リボルビング信用枠か *Y = はい、N = いいえ
    data["RevLineCr"] = data["RevLineCr"].apply(
        lambda x: 1 if x == "Y" else 0 if x == "N" else np.nan
    )
    # 15 万ドル未満のローンを 1 ページの短い申請で処理できるプログラムか *Y = はい、N = いいえ
    data["LowDoc"] = data["LowDoc"].apply(
        lambda x: 1 if x == "Y" else 0 if x == "N" else np.nan
    )
    # Sector 31~33	製造業, 44~45	小売業, 48~49	運輸業、倉庫業
    data["Sector"] = data["Sector"].map({32: 31, 33: 31, 45: 44, 49: 48})
    data = pd.get_dummies(data, columns=["Sector"])
    # City
    MIS_City_list = [
        "BLACKLICK",
        "SCOTTSDALE",
        "SAN FRANCISCO",
        "RALEIGH",
        "HIALEAH",
        "BRUNSWICK",
        "ESCONDIDO",
        "CLARENCE",
        "REHOBOTH",
        "BRAWLEY",
    ]
    data["City"] = data["City"].apply(lambda x: x if x in MIS_City_list else np.nan)
    data = pd.get_dummies(data, columns=["City"])
    # Stateは削除しちゃう
    data = data.drop(["State"], axis=1)
    # $のやつ
    data["DisbursementGross"] = data["DisbursementGross"].str.replace("$", "").str.replace(",", "").str.replace(
        " ", ""
    ).astype(float)
    data["GrAppv"] = data["GrAppv"].str.replace("$", "").str.replace(",", "").str.replace(
        " ", ""
    ).astype(float)
    data["SBA_Appv"] = data["SBA_Appv"].str.replace("$", "").str.replace(",", "").str.replace(
        " ", ""
    ).astype(float)
    # UrbanRural *1 = 都市部、2 = 田舎、0 = 未定義, 都会っぽい順に大きくしてみる
    data["UrbanRural"] = data["UrbanRural"].map({1: 2, 2: 0, 0: 1})

    print(f"********* 加工後のデータshape{data.shape} *********")
    return data

In [ ]:
train = pd.read_csv('s3://xianglishan-sandbox/signate-fuda2/train.csv', index_col=0)
first_column = train.pop('MIS_Status')
train.insert(0,'MIS_Status',first_column)  # MIS_Status はターゲットなので先頭にしておく
test  = pd.read_csv('s3://xianglishan-sandbox/signate-fuda2/test.csv', index_col=0)

print('train')
train_preped = preprocess(train)

print('\ntest')
test_preped  = preprocess(test)

---

In [ ]:
import numpy as np
import pandas as pd


def preprocess(data: pd.DataFrame()) -> pd.DataFrame():
    print(f"********* もともとのデータshape:{data.shape} *********")

    # $のやつ
    def deal_doller(data: pd.DataFrame) -> pd.DataFrame:
        data["DisbursementGross"] = data["DisbursementGross"].str.replace("$", "").str.replace(",", "").str.replace(
            " ", ""
        ).astype(float)
        data["GrAppv"] = data["GrAppv"].str.replace("$", "").str.replace(",", "").str.replace(
            " ", ""
        ).astype(float)
        data["SBA_Appv"] = data["SBA_Appv"].str.replace("$", "").str.replace(",", "").str.replace(
            " ", ""
        ).astype(float)
        return data
    data = deal_doller(data)


    # いろいろ特徴量作成を追加する
    def make_features(data: pd.DataFrame()) -> pd.DataFrame():
        data = data.copy()
        # 新規ビジネスかどうか
        data["NewExist"] = data["NewExist"].map({1: 0, 2: 1})
        # フランチャイズコードは意味なさそう
        data["FranchiseCode"] = data["FranchiseCode"].apply(
            lambda x: 0 if x in (0, 1) else 1
        )
        # リボルビング信用枠か *Y = はい、N = いいえ
        data["RevLineCr"] = data["RevLineCr"].apply(
            lambda x: 1 if x == "Y" else 0 if x == "N" else np.nan
        )
        # 15 万ドル未満のローンを 1 ページの短い申請で処理できるプログラムか *Y = はい、N = いいえ
        data["LowDoc"] = data["LowDoc"].apply(
            lambda x: 1 if x == "Y" else 0 if x == "N" else np.nan
        )
        # count encoding
        #   Sector 31~33	製造業, 44~45	小売業, 48~49	運輸業、倉庫業 -> one hot
        data["Sector"] = data["Sector"].replace({32: 31, 33: 31, 45: 44, 49: 48})
        for col in ['UrbanRural', 'State', 'BankState', 'City', 'Sector']:
            count_dict = dict(data[col].value_counts())
            data[f'{col}_count_encoding'] = data[col].map(count_dict)
            data[f'{col}_count_encoding'] = data[col].map(count_dict).fillna(1).astype(int)
        return data
    
    data = make_features(data)

    print(f"********* 加工後のデータshape{data.shape} *********")
    return data


In [ ]:
train = pd.read_csv('s3://xianglishan-sandbox/signate-fuda2/train.csv', index_col=0)
first_column = train.pop('MIS_Status')
train.insert(0,'MIS_Status',first_column)  # MIS_Status はターゲットなので先頭にしておく
test  = pd.read_csv('s3://xianglishan-sandbox/signate-fuda2/test.csv', index_col=0)

print('train')
train_preped = preprocess(train)

print('\ntest')
test_preped  = preprocess(test)

In [ ]:
train_preped

---

以下は実験用

In [ ]:
pd.get_dummies(train, columns=['Sector']).columns

In [ ]:
#dfの1行目、2行目の列名を連結した列名に変換する関数
def rename_multicol(df):
    df_col=df.columns #列名をコピー
    df = df.T.reset_index(drop=False).T #一回列名をリセット
    for  i in range(df.shape[1]): #列名を新たに定義
        rename_col = {i:"".join(df_col[i])}
        df = df.rename(columns = rename_col)     
    df = df.drop(["level_0","level_1"],axis=0)
    return df

df_temp = rename_multicol(train.groupby('City').agg({'MIS_Status':["sum", "count"]}).reset_index())
df_temp['MIS_ratio'] = 1 - (df_temp['MIS_Statussum'] / df_temp['MIS_Statuscount'])
df_temp.query('MIS_Statuscount>=100 & MIS_ratio>=0.2')\
    .sort_values('MIS_ratio')['City'].tolist()

In [ ]:
train['DisbursementGross'].str.replace("$", "")\
                            .str.replace(',', '')\
                            .str.replace(' ', '')\
                            .astype(float)
# $